In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=jpfoxp2AAXNmT2T6oBPkRf6Bd7aRSlFRoC9RR21J1dU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7akhgLbyXUIchARAoRk14Pds1ziJ3TXOVvVdg2wvOdy5Whshpez1g

Successfully saved authorization token.


In [3]:
#sentinel-2A image collection
S2A = ee.ImageCollection("COPERNICUS/S2_SR")

In [4]:
#date ranges about a month before and after the fire to ensure the availability of cloud free imagery
before_initial = '2019-09-01'
before_final = '2019-10-09'
after_initial = '2019-10-15'
after_final = '2019-11-11'

In [5]:
#A polygon around the city of Calimesa California
roi = ee.Geometry.Polygon([-117.138414, 34.012680, -116.987352, 34.013182, -116.980142, 33.962088, -117.137727, 33.962235])

In [6]:
#Filter the image collection to get the images that intersect with the region of interest
S2A_roi = S2A.filterBounds(roi)

In [7]:
#filter image collection to specified date range
S2A_roi_before = S2A_roi.filterDate(before_initial, before_final)
S2A_roi_after = S2A_roi.filterDate(after_initial, after_final)

In [8]:
#Function to mask clouds using the Sentinel-2 QA band.
def maskS2clouds(image):
  qa = image.select('QA60')

  #Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  #Both flags should be set to zero, indicating clear conditions.
  mask1 = qa.bitwiseAnd(cloudBitMask).eq(0)
  mask = qa.bitwiseAnd(cirrusBitMask).eq(0)

  #Return the masked and scaled data, without the QA bands.
  return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

In [9]:
#Filter the imagery to get only imagery that is relatively cloud free
S2A_roi_before_mask = S2A_roi_before.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)).map(maskS2clouds)
S2A_roi_after_mask = S2A_roi_after.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)).map(maskS2clouds)

In [10]:
#take the median value for the relatively cloud free imagery
S2A_before_image = S2A_roi_before_mask.median()
S2A_afer_image = S2A_roi_after_mask.median()

In [11]:
#calculate the normalized burn ration for both images
before_NBR = S2A_before_image.normalizedDifference(['B8', 'B12'])
after_NBR = S2A_afer_image.normalizedDifference(['B8', 'B12'])

In [12]:
#Take the difference of the NBRs
dNBR = before_NBR.subtract(after_NBR)

In [13]:
#clip to the region of interest
dNBR_clip = dNBR.clip(roi)

In [14]:
#create a boolean layer of burned and non-burned using a threshold of 0.1 as recommended by https://un-spider.org/advisory-support/recommended-practices/recommended-practice-burn-severity/in-detail/normalized-burn-ratio
burned_mask = dNBR_clip.gte(0.1)

In [15]:
#Change all small areas that are categorized as burned to non-burned as they are likely just errors or noise and be left with just the area of the sandalwood fire. Well, almost.
connections = burned_mask.connectedPixelCount()
connection_mask = connections.lt(100)
burned_mask = burned_mask.where(connection_mask, 0)

In [16]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [17]:
#Create a map centered and zoomed in on the region of interest displaying burned area and dNBR
m = folium.Map(location=[33.98, -117.05], zoom_start=13)
m.add_ee_layer(dNBR_clip, {'min': -0.5, 'max': 1.3}, 'dNBR')
m.add_ee_layer(burned_mask, {'min': 0, 'max': 1}, 'Burned')
m.add_child(folium.LayerControl())
display(m)

In [18]:
#create an image where pixel values are the area (meters) of the pixels
Pixel_Area = burned_mask.pixelArea()

In [19]:
#create an image where pixel values are the pixel area for burned pixels and zero for non-burned pixels
Pixel_Area_Burned = Pixel_Area.multiply(burned_mask)

In [20]:
#Use a reducer to calculate the total burned area
burned_area_m = Pixel_Area_Burned.reduceRegion(ee.Reducer.sum(), roi, 20)

In [21]:
#Show total burned area
burned_area_m.getInfo()

{'area': 3985554.644264252}